In [8]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Improving-critical-exponents_pytorch/')
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [11]:
args_cnn = {
    'save_dir': './saved_models/cnn_regression',
    'lattice_size': 128,
    'dataset_size': 5000,
    'epochs': 100,
    'batch_size': 256,
    'dropout_rate': 0.0,
    'learning_rate': 10e-4,
    'device' : device,
    'save_checkpoints': True,
}
        
args_cnn = objectview(args_cnn)

In [ ]:
from src.cnn.train import main as main_cnn
main_cnn(args_cnn)

In [ ]:
args_gen = {
    'save_dir': './saved_models/gan_cnn_regression',
    'epochs': 10,
    'batch_size': 5,
    'learning_rate': 10e-3,
    'device' : device,
    'bins_number': 100,
    'noise_dim': 100,
    'ckpt_freq': 10,
    'CNN_model_path': './saved_models/cnn_regression/2021.10.17.22.40.19/model/final_model.pt',
    'set_generate_plots': False,
    'regularization_strength': 0.5,
}

args_gen = objectview(args_gen)

In [ ]:
from src.generator.train import main as main_gen
main_gen(args_gen)